<a href="https://colab.research.google.com/github/nalpata/proyecto_aplicado_preservantes/blob/main/notebooks/Proyecto_1_Hito_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# HITO 1. RAG Baseline- Proyecto Aplicado: Preservantes

En este notebook construimos el baseline de un sistema RAG usando un conjunto de PDFs
sobre preservantes. Incluye:

1. Carga e ingesta de PDFs
2. Preprocesamiento básico y chunking
3. Generación de embeddings
4. Creación de un vector store
5. Retriever (similarity search)
6. Benchmark (Precision@k sobre un set de preguntas)


In [1]:
## Instalación de librerías (celda de código)
!pip install -q langchain langchain-community langchain-text-splitters \
               chromadb sentence-transformers pypdf


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 7.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 98.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.7/21.7 MB 116.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 328.2/328.2 kB 31.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.2/278.2 kB 27.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 101.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 70.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 476.0/476.0 kB 45.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.3/103.3 kB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.4/17.4 MB 128.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.5/72.5 kB 7.7 MB/s eta

In [2]:
##Importaciones y configuración básica
import os
from pathlib import Path

# LangChain imports
from langchain_community.document_loaders import DirectoryLoader, PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma
from langchain_community.embeddings import HuggingFaceEmbeddings

# Para evaluación básica
from typing import List, Dict
import numpy as np

# Para ver resultados
from pprint import pprint


In [3]:
# RESETEAR TODO PARA PARTIR LIMPIO EN COLAB

import os, shutil

# 1) Ir a /content
%cd /content

# 2) Borrar cualquier clone previo duplicado
if os.path.exists("proyecto_aplicado_preservantes"):
    shutil.rmtree("proyecto_aplicado_preservantes")
    print("🗑️ Carpeta borrada: proyecto_aplicado_preservantes")

# 3) Clonar de nuevo desde tu GitHub
!git clone https://github.com/nalpata/proyecto_aplicado_preservantes.git

# 4) Entrar a la carpeta correcta
%cd proyecto_aplicado_preservantes

print("\n🎉 Listo. Ahora estamos en el repo correcto sin duplicados.")
!ls


/content
Cloning into 'proyecto_aplicado_preservantes'...
remote: Enumerating objects: 359, done.
remote: Counting objects: 100% (175/175), done.
remote: Compressing objects: 100% (140/140), done.
remote: Total 359 (delta 112), reused 32 (delta 32), pack-reused 184 (from 5)
Receiving objects: 100% (359/359), 19.78 MiB | 17.00 MiB/s, done.
Resolving deltas: 100% (166/166), done.
/content/proyecto_aplicado_preservantes

🎉 Listo. Ahora estamos en el repo correcto sin duplicados.
 app.py     'Pauta proyecto.pdf'    requirements.txt
 data	     PROBLEMAS_COMUNES.md   RESUMEN_PROBLEMAS.md
 notebooks   readme.md		    src


In [4]:
# Ruta base del proyecto en Colab
BASE_PATH = Path("/content/proyecto_aplicado_preservantes")

DATA_PDF_DIR = BASE_PATH / "data" / "pdfs"          # aquí PDFs de preservantes
CHROMA_DIR   = BASE_PATH / "chroma_preservantes"   # carpeta donde se guardará el vector store

BASE_PATH.mkdir(parents=True, exist_ok=True)
CHROMA_DIR.mkdir(parents=True, exist_ok=True)

print("Base path:", BASE_PATH)
print("PDF dir:", DATA_PDF_DIR)
print("Chroma dir:", CHROMA_DIR)


Base path: /content/proyecto_aplicado_preservantes
PDF dir: /content/proyecto_aplicado_preservantes/data/pdfs
Chroma dir: /content/proyecto_aplicado_preservantes/chroma_preservantes


In [5]:
##Carga de documentos (ingesta de PDFs)
def load_pdfs(pdf_dir: Path):
    """
    Carga todos los PDFs de una carpeta usando LangChain.
    Devuelve una lista de Documents.
    """
    loader = DirectoryLoader(
        str(pdf_dir),
        glob="*.pdf",
        loader_cls=PyPDFLoader,
        show_progress=True
    )
    docs = loader.load()
    return docs

raw_docs = load_pdfs(DATA_PDF_DIR)
len(raw_docs), raw_docs[0]


100%|██████████| 19/19 [00:21<00:00,  1.14s/it]


(475,
 Document(metadata={'producer': 'Acrobat Distiller 8.1.0 (Windows)', 'creator': 'Elsevier', 'creationdate': '2025-11-03T03:51:39+00:00', 'crossmarkdomains[1]': 'elsevier.com', 'creationdate--text': '3rd November 2025', 'robots': 'noindex', 'elsevierwebpdfspecifications': '7.0.1', 'moddate': '2025-11-03T04:07:06+00:00', 'doi': '10.1016/j.fbio.2025.107864', 'title': 'Mechanisms, applications and challenges of natural antimicrobials in food system', 'keywords': 'Natural antimicrobials,Food preservation,Bioactive compounds,Food safety,Clean label', 'subject': 'Food Bioscience, 74 (2025) 107864. doi:10.1016/j.fbio.2025.107864', 'crossmarkdomains[2]': 'sciencedirect.com', 'author': 'Anand Kumar', 'source': '/content/proyecto_aplicado_preservantes/data/pdfs/1-s2.0-S2212429225020413-main.pdf', 'total_pages': 20, 'page': 0, 'page_label': '1'}, page_content='Mechanisms, applications and challenges of natural antimicrobials in \nfood system\nAnand Kumar\na , 1\n, Suprativ Das\nb , 1\n, Sada

In [6]:
##Preprocesamiento
def clean_metadata(docs):
    """
    Normaliza  los metadatos: agrega un campo 'source'
    y mantiene solo lo relevante.
    """
    cleaned = []
    for d in docs:
        meta = d.metadata or {}
        source = meta.get("source", "")
        # Nos quedamos con un metadata simple
        new_meta = {
            "source": source,
            "page": meta.get("page", None)
        }
        d.metadata = new_meta
        cleaned.append(d)
    return cleaned

docs = clean_metadata(raw_docs)
len(docs), docs[0].metadata


(475,
 {'source': '/content/proyecto_aplicado_preservantes/data/pdfs/1-s2.0-S2212429225020413-main.pdf',
  'page': 0})

**Chunking baseline**

Usamos RecursiveCharacterTextSplitter con:
- chunk_size = 800
- chunk_overlap = 200



In [49]:
##Chunking
##Usamos RecursiveCharacterTextSplitter como baseline.

CHUNK_SIZE = 800
CHUNK_OVERLAP = 200

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=CHUNK_SIZE,
    chunk_overlap=CHUNK_OVERLAP,
    length_function=len,
)

chunks = text_splitter.split_documents(docs)
len(chunks), chunks[0]


(2790,
 Document(metadata={'source': '/content/proyecto_aplicado_preservantes/data/pdfs/1-s2.0-S2212429225020413-main.pdf', 'page': 0}, page_content='Mechanisms, applications and challenges of natural antimicrobials in \nfood system\nAnand Kumar\na , 1\n, Suprativ Das\nb , 1\n, Sadaqat Ali\na\n, Swapnil Ganesh Jaiswal\nc\n,  \nAhmad Rabbani\nd\n, Syed Mohammad Ehsanur Rahman\ne , f\n, Ramachandran Chelliah\nf\n,  \nDeog-Hwan Oh\nf\n, Shucheng Liu\na , g\n, Shuai Wei\na , g , *\na\nCollege of Food Science and Technology, Guangdong Ocean University, Guangdong Provincial Key Laboratory of Aquatic Products Processing and Safety, Guangdong \nProvince Engineering Laboratory for Marine Biological Products, Guangdong Provincial Engineering Technology Research Center of Seafood, Key Laboratory of Advanced \nProcessing of Aquatic Product of Guangdong Higher Education Institution, Zhanjiang, 524088, China\nb'))

In [50]:
##cuántos PDFs y de qué archivo vienen los chunks
from collections import Counter

print("N° de documentos originales:", len(raw_docs))
print("Fuentes (PDFs) originales:")
for src in sorted({d.metadata.get("source") for d in raw_docs}):
    print(" -", src)

print("\nN° de chunks:", len(chunks))
print("N° de chunks por PDF:")
conteo = Counter(d.metadata.get("source") for d in chunks)
for src, c in conteo.items():
    print(f"{src}: {c}")


N° de documentos originales: 475
Fuentes (PDFs) originales:
 - /content/proyecto_aplicado_preservantes/data/pdfs/1-s2.0-S2212429225020413-main.pdf
 - /content/proyecto_aplicado_preservantes/data/pdfs/1-s2.0-S2405844023042287-main.pdf
 - /content/proyecto_aplicado_preservantes/data/pdfs/AntimicrobialActivityofSpiceextracts.pdf
 - /content/proyecto_aplicado_preservantes/data/pdfs/Effects of Acidification and Preservatives on Microbial Growth Puree.pdf
 - /content/proyecto_aplicado_preservantes/data/pdfs/FTB-61-212.pdf
 - /content/proyecto_aplicado_preservantes/data/pdfs/Food aditives1.pdf
 - /content/proyecto_aplicado_preservantes/data/pdfs/IJFS2018-8410747.pdf
 - /content/proyecto_aplicado_preservantes/data/pdfs/Molecules to food, preservatives.pdf
 - /content/proyecto_aplicado_preservantes/data/pdfs/Preservantes.pdf
 - /content/proyecto_aplicado_preservantes/data/pdfs/Prop Ca y s. potasio.pdf
 - /content/proyecto_aplicado_preservantes/data/pdfs/The Scientific World Journal - 2022 - Tes

Usamos chunking gerarquico porque los chunks no estan balanceados y se genera un pdf dominante

In [51]:
## Uso chunking gerarquico
from langchain_text_splitters import RecursiveCharacterTextSplitter
from uuid import uuid4

# Splitter de nivel alto (bloques grandes)
high_level_splitter = RecursiveCharacterTextSplitter(
    chunk_size=2000,
    chunk_overlap=200,
    length_function=len,
)

# Splitter de nivel bajo (para el vector store)
low_level_splitter = RecursiveCharacterTextSplitter(
    chunk_size=700,
    chunk_overlap=150,
    length_function=len,
)

def hierarchical_chunk(docs):
    """
    1. Divide en bloques grandes (nivel 1)
    2. Cada bloque grande se subdivide en chunks pequeños (nivel 2)
    3. Añade metadatos de jerarquía (parent_id, level1_index)
    """
    level1_docs = high_level_splitter.split_documents(docs)

    final_chunks = []
    for idx, d in enumerate(level1_docs):
        parent_id = str(uuid4())  # id único del bloque grande

        # subdividir este bloque
        sub_docs = low_level_splitter.split_documents([d])

        for s in sub_docs:
            meta = dict(s.metadata)
            meta["parent_id"] = parent_id
            meta["level1_index"] = idx
            s.metadata = meta
            final_chunks.append(s)

    return final_chunks

hier_chunks = hierarchical_chunk(docs)
len(hier_chunks), hier_chunks[0].metadata


(3519,
 {'source': '/content/proyecto_aplicado_preservantes/data/pdfs/1-s2.0-S2212429225020413-main.pdf',
  'page': 0,
  'parent_id': '3bb0619c-c06e-4e99-9102-f19c3c5b4a54',
  'level1_index': 0})

**Modelo de embeddings**

In [10]:
EMBEDDING_MODEL_NAME = "sentence-transformers/distiluse-base-multilingual-cased-v2"

embeddings = HuggingFaceEmbeddings(
    model_name=EMBEDDING_MODEL_NAME
)

print("Modelo cargado:", EMBEDDING_MODEL_NAME)


/tmp/ipython-input-990023370.py:3: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the `langchain-huggingface package and should be used instead. To use it run `pip install -U `langchain-huggingface` and import as `from `langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(
/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/341 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/114 [00:00<?, ?B/s]

2_Dense/model.safetensors:   0%|          | 0.00/1.58M [00:00<?, ?B/s]

Modelo cargado: sentence-transformers/distiluse-base-multilingual-cased-v2


**Vector Store**

In [12]:
vector_store = Chroma.from_documents(
    documents=chunks,
    embedding=embeddings,
    persist_directory=str(CHROMA_DIR)
)

vector_store.persist()
print("Vector store creado con", vector_store._collection.count(), "documentos")


Vector store creado con 2790 documentos


/tmp/ipython-input-573959463.py:7: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  vector_store.persist()


In [14]:
from pathlib import Path
from langchain_community.vectorstores import Chroma

BASE_PATH = Path("/content/proyecto_aplicado_preservantes")
CHROMA_HIER_DIR = BASE_PATH / "chroma_preservantes_hier"

import shutil
shutil.rmtree(CHROMA_HIER_DIR, ignore_errors=True)
CHROMA_HIER_DIR.mkdir(parents=True, exist_ok=True)

vector_store_hier = Chroma.from_documents(
    documents=hier_chunks,
    embedding=embeddings,                  # el mismo modelo multilingüe
    persist_directory=str(CHROMA_HIER_DIR)
)

vector_store_hier.persist()
print("Vector store HIER creado con", vector_store_hier._collection.count(), "documentos")


Vector store HIER creado con 3519 documentos


In [15]:
CHROMA_DIR.mkdir(parents=True, exist_ok=True)
#Cuando uso el jerárquico
CHROMA_HIER_DIR.mkdir(parents=True, exist_ok=True)


In [16]:
retriever_hier = vector_store_hier.as_retriever(
    search_type="mmr",
    search_kwargs={"k": 5, "fetch_k": 20}
)


In [17]:
def inspeccionar_query_con(retriever, query: str, k: int = 5):
    docs = retriever.invoke(query)[:k]
    print("Query:", query, "\n")
    for i, d in enumerate(docs, 1):
        print(f"--- Documento {i} ---")
        print("Source:", d.metadata.get("source"), "| Page:", d.metadata.get("page"),
              "| level1_index:", d.metadata.get("level1_index"))
        print(d.page_content[:500], "...\n")

inspeccionar_query_con(retriever_hier, "¿What are the antimicrobial effects of sodium benzoate, sodium nitrite, and potassium sorbate?")


Query: ¿What are the antimicrobial effects of sodium benzoate, sodium nitrite, and potassium sorbate? 

--- Documento 1 ---
Source: /content/proyecto_aplicado_preservantes/data/pdfs/Molecules to food, preservatives.pdf | Page: 3 | level1_index: 240
teria, and fungi. It acts through membrane disruption and inhi-
bition of metabolic reactions, stress, and accumulation of toxic
anions inside the microbial cell (Brul and Coote1999). It may
be coupled to calcium, potassium, or sodium for different an-
timicrobial targets and effects. The main applications of sodium ...

--- Documento 2 ---
Source: /content/proyecto_aplicado_preservantes/data/pdfs/Effects of Acidification and Preservatives on Microbial Growth Puree.pdf | Page: 7 | level1_index: 112
usedalone.Forinstance,antimicrobialactivityagainst E.coli
w a se nh a n c e dth r o u ghth ec o m b i n e du seo f0 . 1%po ta s s i u m
sorbateand0.1%sodiumbenzoateat8
∘Cwi thsurvi valtime
being reduced by 50 % compared with the one with 0.1 %
sod

In [18]:
from langchain_community.vectorstores import Chroma

vector_store_hier = Chroma.from_documents(
    documents=hier_chunks,    # chunks jerárquicos
    embedding=embeddings      # mismo modelo multilingüe
)

print("Vector store jerárquico creado en memoria con:",
      vector_store_hier._collection.count(), "chunks")


Vector store jerárquico creado en memoria con: 3519 chunks


**Retriever jerarquico**

In [19]:
# Retriever jerárquico
retriever_hier = vector_store_hier.as_retriever(
    search_type="mmr",
    search_kwargs={"k": 5, "fetch_k": 20}
)

query_ejemplo = "¿Qué es un preservante y qué función cumple en alimentos?"
resultados_hier = retriever_hier.invoke(query_ejemplo)

len(resultados_hier), resultados_hier[0]


(5,
 Document(metadata={'page': 60, 'source': '/content/proyecto_aplicado_preservantes/data/pdfs/Food aditives1.pdf', 'level1_index': 228, 'parent_id': 'c7b45b22-c1bf-4afd-b61a-d7ff24701e0d'}, page_content='food additives.\n14. What is the importance of gels in processed foods? Give two examples\nof substances used to form gels in food systems.\n15. What is rum caviar and how is it made?\n16. Give two examples of emulsiﬁers and describe their use in food systems.\n17. What is a fat replacer? Give an example.\n18. Give an example of enzymes used as a food additives.\n19. Deﬁne the terms toxin and toxicant. Give an example of each.\n20. Give an example of a toxicant that results from the Maillard reaction.\n21. What is trypsin inhibitor and where is it found?\n22. Name 2 sources of trypsin inhibitor.\n23. What is the difference between direct and indirect food additives?'))

In [20]:
for i, d in enumerate(resultados_hier, 1):
    print(f"\n### Documento {i} ###")
    print("Source:", d.metadata.get("source"), "| Page:", d.metadata.get("page"))
    print(d.page_content[:300], "...\n")



### Documento 1 ###
Source: /content/proyecto_aplicado_preservantes/data/pdfs/Food aditives1.pdf | Page: 60
food additives.
14. What is the importance of gels in processed foods? Give two examples
of substances used to form gels in food systems.
15. What is rum caviar and how is it made?
16. Give two examples of emulsiﬁers and describe their use in food systems.
17. What is a fat replacer? Give an example ...


### Documento 2 ###
Source: /content/proyecto_aplicado_preservantes/data/pdfs/Food aditives1.pdf | Page: 16
and reduction of nutritional value. Additives included in processed foods
perform their antioxidant function either as free radical scavengers or che-
lators of pro-oxidant metal ions.
What are antioxidants? How do they work?
Antioxidants are compounds that inhibit or terminate free radical reaction ...


### Documento 3 ###
Source: /content/proyecto_aplicado_preservantes/data/pdfs/Prop Ca y s. potasio.pdf | Page: 13
permitiendo la formación de la miga suave, pero no  de 

In [21]:
def evaluate_retriever(retriever, eval_queries, k=5, nombre="Evaluación"):
    print(f"\n=== Evaluando retriever: {nombre} ===\n")

    scores = []

    for item in eval_queries:
        query = item["query"]
        keywords = item["relevant_keywords"]

        # Recuperar documentos
        docs = retriever.invoke(query)[:k]

        # Precision@k manual
        hits = 0
        for doc in docs:
            text = doc.page_content.lower()
            if any(kw.lower() in text for kw in keywords):
                hits += 1

        precision = hits / k
        scores.append(precision)

        print(f"Query: {query}")
        print(f"Precision@{k}: {precision:.2f}\n")

    print(f"Precision@{k} promedio: {sum(scores)/len(scores):.2f}")
    return scores


**Naive RAG**

In [22]:
retriever = vector_store.as_retriever(
    search_type="similarity",
    search_kwargs={"k": 5}
)


In [23]:
##Baseline: solo mostrar textos recuperados
def show_retrieval(query: str, k: int = 5, retriever=retriever):
    # Con LangChain nuevo el retriever se invoca así:
    docs = retriever.invoke(query)
    docs = docs[:k]

    print(f"Query: {query}\n")
    for i, d in enumerate(docs, start=1):
        print(f"--- Documento {i} ---")
        print("Source:", d.metadata.get("source"), "Page:", d.metadata.get("page"))
        print(d.page_content[:500], "...")
        print()

# Prueba
show_retrieval("Tipos de preservantes utilizados en bebidas", retriever=retriever_hier)


Query: Tipos de preservantes utilizados en bebidas

--- Documento 1 ---
Source: /content/proyecto_aplicado_preservantes/data/pdfs/j.ijfoodmicro.2013.06.025.pdf Page: 1
USA
6.55 8 533
6 Z. bailii Spoilage, bottled ice tea
USA
7.46 9.12 545
7 Z. bailii Spoilage, preserved
fruit punch USA
6.67 8.13 475
8 Z. bailii Spoilage, soft drink USA 6.68 8.5 467
9 Z. bailii Spoilage, carbonated
orange drink USA
8.04 8.13 468
10 Z. bailii Spoilage, soft drink USA 6.35 8.33 483
11 Z. bailii Spoilage, soft drink USA 7 9.13 466
12 Z. bailii Spoilage, carbonated
orange drink USA
8.09 9.75 468
13 Z. bailii Spoilage, soft drink USA 7.06 10.12 467
15 Z. bailii Spoilage, salad dress ...

--- Documento 2 ---
Source: /content/proyecto_aplicado_preservantes/data/pdfs/Prop Ca y s. potasio.pdf Page: 149
ANEXO C
HOJA DE CATACIÓN
DIAGRAMA DE FLUJO 
143 ...

--- Documento 3 ---
Source: /content/proyecto_aplicado_preservantes/data/pdfs/Food aditives1.pdf Page: 9
wine, fruit juices, and some baked goods (pastries). It

**Integramos un LLM para responder**

In [ ]:
!pip install -q langchain-openai langchain-community openai tiktoken


In [ ]:
!pip install -q langchain langchain-openai langchain-community langchain-text-splitters
!pip install -q langchain-core
!pip install -q langchain-experimental
!pip install -q langchainhub
!pip install -q lc-retrieval


In [ ]:
import os
import getpass

from langchain_openai import ChatOpenAI
from langchain_core.runnables import RunnablePassthrough


In [ ]:
os.environ["OPENAI_API_KEY"] = getpass.getpass("Ingresa tu OPENAI_API_KEY: ")


In [ ]:
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.1)


In [ ]:
def ask_rag(query: str, k: int = 5, retriever=retriever_hier):
    # 1. Recuperar documentos relevantes
    docs = retriever.invoke(query)
    docs = docs[:k]

    # 2. Construir el contexto a partir de los chunks
    context = "\n\n---\n\n".join(d.page_content for d in docs)

    # 3. Armar el prompt para el LLM
    prompt = f"""
Eres un asistente experto en preservantes de alimentos.
Responde usando EXCLUSIVAMENTE la información del contexto.

Contexto:
{context}

Pregunta: {query}

Respuesta en español, clara y concisa:
"""

    # 4. Llamar al modelo
    response = llm.invoke(prompt)

    # 5. Mostrar resultado y fuentes
    print("Pregunta:", query)
    print("\n Respuesta:\n")
    print(response.content)

    print("\n Fuentes:")
    for d in docs:
        print("-", d.metadata.get("source"), "| page", d.metadata.get("page"))


In [ ]:
ask_rag("Tipos de preservantes utilizados en bebidas", retriever=retriever_hier)


**Benchmark  (Precision@k)**

In [24]:
retriever_hier = vector_store_hier.as_retriever(
    search_type="mmr",          # búsqueda diversificada
    search_kwargs={
        "k": 5,                 # número final de documentos que regresará
        "fetch_k": 20           # número de documentos que explora primero
    }
)


In [25]:
eval_queries = [
{
  "query": "¿Qué es un preservante antimicrobiano?",
  "relevant_keywords": [
    "preservante antimicrobiano",
    "conservante antimicrobiano",
    "inhibición microbiana",
    "inhibe el crecimiento microbiano",
    "sustancia antimicrobiana",
    "agente antimicrobiano",
    "inhibición de microorganismos",

    "antimicrobial preservative",
    "antimicrobial agent",
    "microbial growth inhibition",
    "inhibits microbial growth"
  ]
},
{
  "query": "¿Cuáles son los factores que afectan la efectividad de los preservantes?",
  "relevant_keywords": [
    "efectividad de los preservantes",
    "factores que afectan la efectividad",
    "actividad de agua",
    "aw",
    "concentración del conservante",
    "concentración inhibitoria",
    "pKa del conservante",
    "interacción con composición del alimento",

    "preservative effectiveness",
    "factors influencing preservative efficacy",
    "water activity",
    "aw value",
    "preservative concentration",
    "food composition interaction",
    "minimum inhibitory concentration"
  ]
},
{
  "query": "¿Qué se entiende por vida útil de un alimento?",
  "relevant_keywords": [
    "vida útil del alimento",
    "vida útil",
    "deterioro microbiano",
    "estabilidad del alimento",
    "seguridad alimentaria",
    "calidad durante el almacenamiento",

    "shelf life",
    "food shelf life",
    "food spoilage",
    "microbial spoilage",
    "quality stability",
    "storage stability"
  ]
}
]


In [26]:
scores_hier = evaluate_retriever(
    retriever_hier,
    eval_queries,
    k=5,
    nombre="Jerárquico (MMR + chunking estructural)"
)



=== Evaluando retriever: Jerárquico (MMR + chunking estructural) ===

Query: ¿Qué es un preservante antimicrobiano?
Precision@5: 0.20

Query: ¿Cuáles son los factores que afectan la efectividad de los preservantes?
Precision@5: 0.20

Query: ¿Qué se entiende por vida útil de un alimento?
Precision@5: 0.40

Precision@5 promedio: 0.27


In [27]:
from typing import List
import numpy as np

def precision_at_k(query: str, retrieved_docs: List, keywords: List[str], k: int = 5):
    """
    Calcula Precision@k verificando si los documentos recuperados contienen keywords relevantes.
    """
    hits = 0
    for doc in retrieved_docs[:k]:
        text = doc.page_content.lower()
        # Si alguna keyword aparece en el texto => HIT
        if any(keyword.lower() in text for keyword in keywords):
            hits += 1

    return hits / k  # Precision@k


def evaluate_retriever_precision(retriever, eval_queries, k: int = 5, nombre: str = "Modelo"):
    """
    Aplica Precision@k a un conjunto de queries y muestra resultados.
    """
    print(f"\n=== Evaluando retriever: {nombre} ===\n")

    scores = []
    for item in eval_queries:
        query = item["query"]
        keywords = item["relevant_keywords"]

        # Recuperar documentos
        retrieved = retriever.invoke(query)

        # Calcular Prec@k
        score = precision_at_k(query, retrieved, keywords, k)
        scores.append(score)

        print(f"Query: {query}")
        print(f"Precision@{k}: {score:.2f}\n")

    print(f"Precision@{k} promedio: {np.mean(scores):.2f}")
    return scores


In [28]:
scores_hier = evaluate_retriever_precision(
    retriever_hier,
    eval_queries,
    k=5,
    nombre="Jerárquico (MMR)"
)



=== Evaluando retriever: Jerárquico (MMR) ===

Query: ¿Qué es un preservante antimicrobiano?
Precision@5: 0.20

Query: ¿Cuáles son los factores que afectan la efectividad de los preservantes?
Precision@5: 0.20

Query: ¿Qué se entiende por vida útil de un alimento?
Precision@5: 0.40

Precision@5 promedio: 0.27


**CONCLUSIONES**

Corpus heterogéneo (inglés/español): Los documentos contienen conceptos relevantes en distintos idiomas, lo que afecta la recuperación cuando la evaluación depende de keywords únicamente en español o traducciones exactas.

Evaluación basada en coincidencia de palabras clave: Precision@k penaliza documentos que son relevantes conceptualmente, pero no contienen literalmente las palabras clave definidas.

Preguntas conceptuales difíciles: Consultas de tipo “¿Qué es…?” requieren definiciones explícitas que pueden no aparecer como tal en el corpus o estar formuladas con vocabulario técnico, reduciendo la recuperación efectiva.

Tamaño y calidad del corpus: Aunque el corpus es valioso, varias fuentes no están estructuradas pedagógicamente y contienen tablas, fórmulas o párrafos extensos, lo que dificulta la segmentación óptima.

**POSIBLES MEJORAS PARA SIGUENTE HITO**

Mejorar los embeddings. Adoptar un modelo más robusto y científico multilingüe

Optimizar el proceso de chunking: Usar chunking híbrido (estructura + semántica + tamaño).

Incluir metadatos explícitos (subtítulos, figuras, secciones) para mejorar contexto jerárquico.

Mejorar la evaluación: Expandir keywords con sinónimos y variaciones técnicas.



In [29]:
# HITO 1 — resultados congelados


BASE_K = 5

baseline_scores = evaluate_retriever_precision(
    retriever,
    eval_queries,
    k=BASE_K,
    nombre="Hito 1 - Baseline Naive"
)

baseline_precision = float(np.mean(baseline_scores))

print("\n BASELINE CONGELADO")
print(f"Precision@{BASE_K} = {baseline_precision:.4f}")



=== Evaluando retriever: Hito 1 - Baseline Naive ===

Query: ¿Qué es un preservante antimicrobiano?
Precision@5: 0.20

Query: ¿Cuáles son los factores que afectan la efectividad de los preservantes?
Precision@5: 0.20

Query: ¿Qué se entiende por vida útil de un alimento?
Precision@5: 0.40

Precision@5 promedio: 0.27

 BASELINE CONGELADO
Precision@5 = 0.2667


In [30]:
import pandas as pd
import os

os.makedirs("results", exist_ok=True)

pd.DataFrame([{
    "modelo": "Hito 1 - Baseline Naive",
    "Precision@5": baseline_precision
}]).to_csv("results/hito1_baseline.csv", index=False)

print(" Baseline guardado en results/hito1_baseline.csv")


 Baseline guardado en results/hito1_baseline.csv


# **HITO 2** RAG Baseline MEJORADO- Proyecto Aplicado: Preservantes

**1 Instalar y definir paths**

In [31]:
!pip -q install langchain langchain-community langchain-text-splitters chromadb sentence-transformers pypdf

import os
from pathlib import Path
import numpy as np
import pandas as pd

BASE_PATH = Path("/content/proyecto_aplicado_preservantes")
CHROMA_HIER_DIR = BASE_PATH / "chroma_preservantes_hier"   # <- el que usaste en Hito 1
RESULTS_DIR = BASE_PATH / "results"
RESULTS_DIR.mkdir(parents=True, exist_ok=True)

print("BASE_PATH:", BASE_PATH)
print("CHROMA_HIER_DIR exists?:", CHROMA_HIER_DIR.exists())
print("RESULTS_DIR:", RESULTS_DIR)


BASE_PATH: /content/proyecto_aplicado_preservantes
CHROMA_HIER_DIR exists?: True
RESULTS_DIR: /content/proyecto_aplicado_preservantes/results


**2.Cargar embeddings**

In [32]:
from langchain_community.embeddings import HuggingFaceEmbeddings

EMBEDDING_MODEL_NAME = "sentence-transformers/distiluse-base-multilingual-cased-v2"
embeddings = HuggingFaceEmbeddings(model_name=EMBEDDING_MODEL_NAME)

print("Embeddings:", EMBEDDING_MODEL_NAME)


Embeddings: sentence-transformers/distiluse-base-multilingual-cased-v2


**3 Cargar el vector store jerárquico**

In [33]:
from langchain_community.vectorstores import Chroma

vector_store_hier = Chroma(
    persist_directory=str(CHROMA_HIER_DIR),
    embedding_function=embeddings,
)

print("Loaded Chroma collection size:", vector_store_hier._collection.count())


Loaded Chroma collection size: 3519


/tmp/ipython-input-921858831.py:3: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the `langchain-chroma package and should be used instead. To use it run `pip install -U `langchain-chroma` and import as `from `langchain_chroma import Chroma``.
  vector_store_hier = Chroma(


**4.Funciones de evaluacion Hito 1**

In [34]:
from typing import List

def precision_at_k(retrieved_docs: List, keywords: List[str], k: int = 5) -> float:
    hits = 0
    for doc in retrieved_docs[:k]:
        text = (doc.page_content or "").lower()
        if any(keyword.lower() in text for keyword in keywords):
            hits += 1
    return hits / k

def evaluate_retriever_precision(retriever, eval_queries, k: int = 5, nombre: str = "Modelo"):
    print(f"\n=== Evaluando retriever: {nombre} ===\n")
    scores = []
    for item in eval_queries:
        query = item["query"]
        keywords = item["relevant_keywords"]
        retrieved = retriever.invoke(query)
        score = precision_at_k(retrieved, keywords, k)
        scores.append(score)
        print(f"Query: {query}")
        print(f"Precision@{k}: {score:.2f}\n")
    print(f"Precision@{k} promedio: {float(np.mean(scores)):.2f}")
    return scores


**5.Definir el set eval_queries**

In [35]:
eval_queries = [
{
  "query": "¿Qué es un preservante antimicrobiano?",
  "relevant_keywords": [
    "preservante antimicrobiano",
    "conservante antimicrobiano",
    "inhibición microbiana",
    "inhibe el crecimiento microbiano",
    "sustancia antimicrobiana",
    "agente antimicrobiano",
    "inhibición de microorganismos",

    "antimicrobial preservative",
    "antimicrobial agent",
    "microbial growth inhibition",
    "inhibits microbial growth"
  ]
},
{
  "query": "¿Cuáles son los factores que afectan la efectividad de los preservantes?",
  "relevant_keywords": [
    "efectividad de los preservantes",
    "factores que afectan la efectividad",
    "actividad de agua",
    "aw",
    "concentración del conservante",
    "concentración inhibitoria",
    "pKa del conservante",
    "interacción con composición del alimento",

    "preservative effectiveness",
    "factors influencing preservative efficacy",
    "water activity",
    "aw value",
    "preservative concentration",
    "food composition interaction",
    "minimum inhibitory concentration"
  ]
},
{
  "query": "¿Qué se entiende por vida útil de un alimento?",
  "relevant_keywords": [
    "vida útil del alimento",
    "vida útil",
    "deterioro microbiano",
    "estabilidad del alimento",
    "seguridad alimentaria",
    "calidad durante el almacenamiento",

    "shelf life",
    "food shelf life",
    "food spoilage",
    "microbial spoilage",
    "quality stability",
    "storage stability"
  ]
}
]


**6 Crear baseline retriever + evaluar + congelar**

In [36]:
BASE_K = 5

retriever_base = vector_store_hier.as_retriever(
    search_type="similarity",
    search_kwargs={"k": BASE_K}
)

scores_base = evaluate_retriever_precision(
    retriever_base,
    eval_queries,
    k=BASE_K,
    nombre="Hito 1/2 - Baseline (similarity sobre hier_chunks)"
)

baseline_precision = float(np.mean(scores_base))
print("\nBASELINE CONGELADO")
print("Precision@5 =", baseline_precision)



=== Evaluando retriever: Hito 1/2 - Baseline (similarity sobre hier_chunks) ===

Query: ¿Qué es un preservante antimicrobiano?
Precision@5: 0.20

Query: ¿Cuáles son los factores que afectan la efectividad de los preservantes?
Precision@5: 0.20

Query: ¿Qué se entiende por vida útil de un alimento?
Precision@5: 0.40

Precision@5 promedio: 0.27

BASELINE CONGELADO
Precision@5 = 0.26666666666666666


In [37]:
out_path = RESULTS_DIR / "hito1_baseline.csv"

pd.DataFrame([{
    "modelo": "Baseline similarity (hier store)",
    "embedding_model": EMBEDDING_MODEL_NAME,
    "k": BASE_K,
    "precision_at_k": baseline_precision,
}]).to_csv(out_path, index=False)

print(" Baseline guardado en:", out_path)


 Baseline guardado en: /content/proyecto_aplicado_preservantes/results/hito1_baseline.csv


**7. MEJORA MMR**

In [38]:
retriever_mmr = vector_store_hier.as_retriever(
    search_type="mmr",
    search_kwargs={"k": 5, "fetch_k": 30, "lambda_mult": 0.5}
)

scores_mmr = evaluate_retriever_precision(
    retriever_mmr,
    eval_queries,
    k=5,
    nombre="Hito 2 - MMR"
)

mmr_precision = float(np.mean(scores_mmr))
print("Precision@5 (MMR):", mmr_precision)
print("Delta vs baseline:", mmr_precision - baseline_precision)



=== Evaluando retriever: Hito 2 - MMR ===

Query: ¿Qué es un preservante antimicrobiano?
Precision@5: 0.20

Query: ¿Cuáles son los factores que afectan la efectividad de los preservantes?
Precision@5: 0.20

Query: ¿Qué se entiende por vida útil de un alimento?
Precision@5: 0.40

Precision@5 promedio: 0.27
Precision@5 (MMR): 0.26666666666666666
Delta vs baseline: 0.0


In [39]:
out_path = RESULTS_DIR / "hito2_mmr.csv"

pd.DataFrame([{
    "modelo": "MMR",
    "embedding_model": EMBEDDING_MODEL_NAME,
    "k": 5,
    "fetch_k": 30,
    "lambda_mult": 0.5,
    "precision_at_k": mmr_precision,
    "delta_vs_baseline": mmr_precision - baseline_precision
}]).to_csv(out_path, index=False)

print(" MMR guardado en:", out_path)


 MMR guardado en: /content/proyecto_aplicado_preservantes/results/hito2_mmr.csv


In [40]:
mmr_configs = [
    {"k": 5, "fetch_k": 20, "lambda_mult": 0.2},
    {"k": 5, "fetch_k": 20, "lambda_mult": 0.5},
    {"k": 5, "fetch_k": 20, "lambda_mult": 0.8},
    {"k": 5, "fetch_k": 50, "lambda_mult": 0.2},
    {"k": 5, "fetch_k": 50, "lambda_mult": 0.5},
    {"k": 5, "fetch_k": 50, "lambda_mult": 0.8},
]

rows = []
for cfg in mmr_configs:
    retriever_mmr = vector_store_hier.as_retriever(
        search_type="mmr",
        search_kwargs=cfg
    )
    scores = evaluate_retriever_precision(
        retriever_mmr,
        eval_queries,
        k=cfg["k"],
        nombre=f"MMR k={cfg['k']} fetch_k={cfg['fetch_k']} lambda={cfg['lambda_mult']}"
    )
    p = float(np.mean(scores))
    rows.append({**cfg, "precision_at_k": p, "delta_vs_baseline": p - baseline_precision})

df_mmr = pd.DataFrame(rows).sort_values("precision_at_k", ascending=False)
df_mmr



=== Evaluando retriever: MMR k=5 fetch_k=20 lambda=0.2 ===

Query: ¿Qué es un preservante antimicrobiano?
Precision@5: 0.00

Query: ¿Cuáles son los factores que afectan la efectividad de los preservantes?
Precision@5: 0.00

Query: ¿Qué se entiende por vida útil de un alimento?
Precision@5: 0.60

Precision@5 promedio: 0.20

=== Evaluando retriever: MMR k=5 fetch_k=20 lambda=0.5 ===

Query: ¿Qué es un preservante antimicrobiano?
Precision@5: 0.20

Query: ¿Cuáles son los factores que afectan la efectividad de los preservantes?
Precision@5: 0.20

Query: ¿Qué se entiende por vida útil de un alimento?
Precision@5: 0.40

Precision@5 promedio: 0.27

=== Evaluando retriever: MMR k=5 fetch_k=20 lambda=0.8 ===

Query: ¿Qué es un preservante antimicrobiano?
Precision@5: 0.20

Query: ¿Cuáles son los factores que afectan la efectividad de los preservantes?
Precision@5: 0.20

Query: ¿Qué se entiende por vida útil de un alimento?
Precision@5: 0.20

Precision@5 promedio: 0.20

=== Evaluando retriever:

,k,fetch_k,lambda_mult,precision_at_k,delta_vs_baseline
1,5,20,0.5,0.266667,0.000000
2,5,20,0.8,0.200000,-0.066667
4,5,50,0.5,0.200000,-0.066667
3,5,50,0.2,0.200000,-0.066667
0,5,20,0.2,0.200000,-0.066667
5,5,50,0.8,0.133333,-0.133333


**CONCLUSION:**

MMR no mejora Precision@5 en este dominio debido a la homogeneidad temática de los documentos y al uso previo de chunking jerárquico

**8. Mejora de Retrieval #1: Query processing (preprocesamiento/expansión)**

In [41]:
#Funciòn de reprocesamiento
def normalize_query(q: str) -> str:
    q = q.lower().strip()
    q = re.sub(r"\s+", " ", q)
    return q

DOMAIN_SYNONYMS = {
    "vida útil": ["shelf life", "duración", "almacenamiento", "estabilidad"],
    "preservante": ["conservante", "aditivo", "preservative"],
    "antimicrobiano": ["antimicrobial", "inhibición microbiana", "microorganismos"],
    "actividad de agua": ["aw", "water activity"],
}

def expand_query(q: str) -> str:
    qn = normalize_query(q)
    extra = []
    for k, syns in DOMAIN_SYNONYMS.items():
        if k in qn:
            extra.extend(syns)
    if extra:
        return q + " | " + " ".join(extra)
    return q


In [42]:
# FIX: regex para normalize_query
import re

def normalize_query(q: str) -> str:
    q = (q or "").lower().strip()
    q = re.sub(r"\s+", " ", q)
    return q


In [43]:
# WRAPPERS DE RETRIEVER

import numpy as np

def _call_retriever(r, query: str):
    """
    Llama al retriever sin asumir si tiene .invoke() o .get_relevant_documents().
    """
    if hasattr(r, "invoke") and callable(getattr(r, "invoke")):
        return r.invoke(query)
    if hasattr(r, "get_relevant_documents") and callable(getattr(r, "get_relevant_documents")):
        return r.get_relevant_documents(query)
    raise AttributeError("El retriever no tiene ni .invoke() ni .get_relevant_documents()")

class QueryExpansionRetriever:
    def __init__(self, base_retriever, expand_fn):
        self.base_retriever = base_retriever
        self.expand_fn = expand_fn

    def invoke(self, query: str):
        q2 = self.expand_fn(query)
        return _call_retriever(self.base_retriever, q2)

    def get_relevant_documents(self, query: str):
        return self.invoke(query)

def evaluate_retriever_precision(retriever, eval_queries, k=5, nombre=""):
    """
    Evalúa Precision@k usando keywords relevantes.
    eval_queries: lista de dicts {"query": str, "relevant_keywords": [..]}
    """
    print(f"\n=== Evaluando retriever: {nombre} ===\n")
    scores = []

    for item in eval_queries:
        query = item["query"]
        keywords = [kw.lower() for kw in item["relevant_keywords"]]

        retrieved = _call_retriever(retriever, query)

        # Normaliza a lista de Document
        if retrieved is None:
            retrieved = []
        if not isinstance(retrieved, list):
            retrieved = list(retrieved)

        topk = retrieved[:k]

        hits = 0
        for doc in topk:
            text = ""
            if hasattr(doc, "page_content"):
                text = (doc.page_content or "").lower()
            else:
                text = str(doc).lower()

            if any(kw in text for kw in keywords):
                hits += 1

        score = hits / max(k, 1)
        scores.append(score)

        print(f"Query: {query}")
        print(f"Precision@{k}: {score:.2f}\n")

    print(f"Precision@{k} promedio: {float(np.mean(scores)):.2f}")
    return scores


# Base
base_ret = vector_store_hier.as_retriever(
    search_type="similarity",
    search_kwargs={"k": BASE_K}
)

# Query expansion
retriever_qproc = QueryExpansionRetriever(base_ret, expand_query)

scores_qproc = evaluate_retriever_precision(
    retriever_qproc,
    eval_queries,
    k=BASE_K,
    nombre="Hito 2 - Query Processing (expansion)"
)

qproc_precision = float(np.mean(scores_qproc))
print("\nP@5 (QueryProc) =", qproc_precision)
print("Delta vs baseline =", qproc_precision - baseline_precision)



=== Evaluando retriever: Hito 2 - Query Processing (expansion) ===

Query: ¿Qué es un preservante antimicrobiano?
Precision@5: 0.20

Query: ¿Cuáles son los factores que afectan la efectividad de los preservantes?
Precision@5: 0.20

Query: ¿Qué se entiende por vida útil de un alimento?
Precision@5: 0.60

Precision@5 promedio: 0.33

P@5 (QueryProc) = 0.3333333333333333
Delta vs baseline = 0.06666666666666665


In [44]:
import os, pandas as pd

os.makedirs("results", exist_ok=True)

row = {
    "modelo": "Hito 2 - Query Processing (expansion)",
    "k": 5,
    "precision_at_5": float(qproc_precision),
    "baseline_precision_at_5": float(baseline_precision),
    "delta_vs_baseline": float(qproc_precision - baseline_precision),
}

df = pd.DataFrame([row])
df.to_csv("results/hito2_query_processing.csv", index=False)

print(df)
print(" Guardado en results/hito2_query_processing.csv")


                                  modelo  k  precision_at_5  \
0  Hito 2 - Query Processing (expansion)  5        0.333333   

   baseline_precision_at_5  delta_vs_baseline  
0                 0.266667           0.066667  
 Guardado en results/hito2_query_processing.csv


**CONCLUSION:**

En promedio, con Query Processing estás logrando que 1 de cada 3 resultados en el top-5 sea relevante (vs 1 de cada 4 en el baseline). Para un set pequeño de queries (3) esto es una señal clara de mejora.

**9. Reranking gratuito usando SentenceTransformers CrossEncoder**

In [45]:
!pip -q install sentence-transformers

import numpy as np
import pandas as pd
import os
import torch
from sentence_transformers import CrossEncoder

# 1) Retriever base: traemos más candidatos (fetch_k)
FETCH_K = 30   # candidatos iniciales
TOP_K   = 5    # lo que devolvemos tras rerank

base_ret_fetch = vector_store_hier.as_retriever(
    search_type="similarity",
    search_kwargs={"k": FETCH_K}
)

# 2) Modelo de reranking (gratis)
RERANK_MODEL_NAME = "cross-encoder/ms-marco-MiniLM-L-6-v2"
device = "cuda" if torch.cuda.is_available() else "cpu"
reranker = CrossEncoder(RERANK_MODEL_NAME, device=device)

# 3) Wrapper retriever con .invoke()
class RerankRetriever:
    def __init__(self, base_retriever, cross_encoder, top_k=5, fetch_k=30):
        self.base = base_retriever
        self.ce = cross_encoder
        self.top_k = top_k
        self.fetch_k = fetch_k

    def _rerank(self, query, docs):
        if len(docs) == 0:
            return []
        pairs = [(query, d.page_content) for d in docs]
        scores = self.ce.predict(pairs)  # array de scores
        idx = np.argsort(scores)[::-1][: self.top_k]
        return [docs[i] for i in idx]

    def invoke(self, query: str):
        # base retriever ya trae fetch_k (por search_kwargs)
        docs = self.base.invoke(query)
        return self._rerank(query, docs)

    # compat por si luego usas otros evaluadores LangChain
    def get_relevant_documents(self, query: str):
        return self.invoke(query)

retriever_rerank = RerankRetriever(
    base_retriever=base_ret_fetch,
    cross_encoder=reranker,
    top_k=TOP_K,
    fetch_k=FETCH_K
)

# 4) Evaluación
scores_rerank = evaluate_retriever_precision(
    retriever_rerank,
    eval_queries,
    k=TOP_K,
    nombre=f"Hito 2 - Reranking (CrossEncoder) fetch_k={FETCH_K}"
)

rerank_precision = float(np.mean(scores_rerank))
delta = rerank_precision - float(baseline_precision)

print("\n RESULTADO RERANKING")
print("Precision@5 (Rerank):", rerank_precision)
print("Delta vs baseline:", delta)

# 5) Guardar resultados (CSV)
os.makedirs("results", exist_ok=True)
df = pd.DataFrame([{
    "modelo": "Hito 2 - Reranking (CrossEncoder ms-marco-MiniLM-L-6-v2)",
    "k": TOP_K,
    "fetch_k": FETCH_K,
    "precision_at_k": rerank_precision,
    "baseline_precision_at_k": float(baseline_precision),
    "delta_vs_baseline": delta,
    "device": device
}])
df.to_csv("results/hito2_reranking.csv", index=False)
print(" Guardado en results/hito2_reranking.csv")
df


config.json:   0%|          | 0.00/794 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/90.9M [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/132 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]


=== Evaluando retriever: Hito 2 - Reranking (CrossEncoder) fetch_k=30 ===

Query: ¿Qué es un preservante antimicrobiano?
Precision@5: 0.00

Query: ¿Cuáles son los factores que afectan la efectividad de los preservantes?
Precision@5: 0.20

Query: ¿Qué se entiende por vida útil de un alimento?
Precision@5: 0.80

Precision@5 promedio: 0.33

 RESULTADO RERANKING
Precision@5 (Rerank): 0.3333333333333333
Delta vs baseline: 0.06666666666666665
 Guardado en results/hito2_reranking.csv


,modelo,k,fetch_k,precision_at_k,baseline_precision_at_k,delta_vs_baseline,device
0,Hito 2 - Reranking (CrossEncoder ms-marco-Mini...,5,30,0.333333,0.266667,0.066667,cuda


**CONCLUSION:**

La incorporación de una etapa de reranking basada en un CrossEncoder permitió mejorar de manera consistente el desempeño del sistema de recuperación de información en comparación con el baseline definido en el Hito 1. En particular, la métrica Precision@5 aumentó desde un valor aproximado de 0.27 en el baseline hasta 0.33 tras aplicar reranking, lo que representa una mejora absoluta de +0.0667.

Este resultado evidencia que, si bien el vector store jerárquico es capaz de recuperar fragmentos relevantes, el orden inicial de los documentos no siempre prioriza aquellos más alineados semánticamente con la intención de la consulta. El reranking actúasobre esta limitación, reevaluando los documentos candidatos mediante un modelo más expresivo que considera de forma conjunta la consulta y cada fragmento recuperado, logrando así un ordenamiento más preciso en el top-k final.

**10.HYBRID SEARCH = BM25 + Hybrid (BM25 + Dense jerárquico)**

In [52]:
from langchain_core.documents import Document

# seguridad: verificar que all_splits exista
assert "all_splits" in globals(), " all_splits no existe. Ejecuta el chunking del Hito 1 primero"

# convertir a Document si fuera string (BM25 lo necesita)
if len(all_splits) > 0 and isinstance(all_splits[0], str):
    all_splits = [Document(page_content=t) for t in all_splits]

print(" all_splits listo. Tipo:", type(all_splits[0]))


AssertionError:  all_splits no existe. Ejecuta el chunking del Hito 1 primero

In [47]:
!pip -q install rank_bm25


In [53]:
from langchain_community.retrievers import BM25Retriever

# 1) Sparse retriever (BM25)
bm25_ret = BM25Retriever.from_documents(all_splits)
bm25_ret.k = BASE_K  # usa el mismo k del baseline (ej: 5)

# 2) Dense retriever (el tuyo: jerárquico)
dense_ret = vector_store_hier.as_retriever(
    search_type="similarity",
    search_kwargs={"k": BASE_K}
)

# 3) Wrapper Hybrid: combina resultados y quita duplicados por (source,page) o por texto
def hybrid_invoke(query: str, k: int = BASE_K):
    # traemos más candidatos para mezclar
    d_docs = dense_ret.get_relevant_documents(query)
    b_docs = bm25_ret.get_relevant_documents(query)

    merged = []
    seen = set()

    for doc in (d_docs + b_docs):
        src = doc.metadata.get("source", "")
        page = doc.metadata.get("page", "")
        key = (src, page, doc.page_content[:200])  # robusto si falta page
        if key not in seen:
            merged.append(doc)
            seen.add(key)

    return merged[:k]

# 4) Adaptador con .invoke() para tu evaluate_retriever_precision
class HybridRetriever:
    def __init__(self, k=BASE_K):
        self.k = k
    def invoke(self, query: str):
        return hybrid_invoke(query, k=self.k)

hybrid_ret = HybridRetriever(k=BASE_K)

# 5) Evaluación BM25 solo
scores_bm25 = evaluate_retriever_precision(
    bm25_ret, eval_queries, k=BASE_K, nombre=f"Hito 2 - BM25 (k={BASE_K})"
)
bm25_precision = float(np.mean(scores_bm25))
print("P@5 (BM25) =", bm25_precision, "Delta =", bm25_precision - baseline_precision)

# 6) Evaluación Hybrid
scores_hybrid = evaluate_retriever_precision(
    hybrid_ret, eval_queries, k=BASE_K, nombre=f"Hito 2 - HYBRID (BM25 + Dense, k={BASE_K})"
)
hybrid_precision = float(np.mean(scores_hybrid))
print("P@5 (HYBRID) =", hybrid_precision, "Delta =", hybrid_precision - baseline_precision)


NameError: name 'all_splits' is not defined

In [54]:
import numpy as np

# --- helper: llamar retrievers de forma compatible (invoke vs get_relevant_documents) ---
def _retrieve_any(retriever, query: str):
    if hasattr(retriever, "invoke"):
        return retriever.invoke(query)
    if hasattr(retriever, "get_relevant_documents"):
        return retriever.get_relevant_documents(query)
    raise AttributeError(f"Retriever sin método compatible: {type(retriever)}")

# --- Hybrid simple por "unión + re-ranking por score" (sin EnsembleRetriever) ---
class HybridUnionRetriever:
    def __init__(self, dense_ret, bm25_ret, k=5):
        self.dense_ret = dense_ret
        self.bm25_ret = bm25_ret
        self.k = k

    def invoke(self, query: str):
        dense_docs = _retrieve_any(self.dense_ret, query)
        bm25_docs  = _retrieve_any(self.bm25_ret, query)

        # unión por texto+source para evitar duplicados
        seen = set()
        merged = []
        for d in (dense_docs + bm25_docs):
            src = (d.metadata or {}).get("source", "")
            key = (src, d.page_content[:200])
            if key not in seen:
                seen.add(key)
                merged.append(d)

        return merged[: self.k]

# --- Evaluación P@k compatible ---
def evaluate_retriever_precision(retriever, eval_queries, k=5, nombre=""):
    scores = []
    print(f"\n=== Evaluando retriever: {nombre} ===\n")
    for item in eval_queries:
        q = item["query"]
        keywords = item["relevant_keywords"]

        retrieved = _retrieve_any(retriever, q)
        retrieved = retrieved[:k]

        # precision@k: cuenta si aparece al menos una keyword en cada doc
        hits = 0
        for doc in retrieved:
            text = (doc.page_content or "").lower()
            if any(kw.lower() in text for kw in keywords):
                hits += 1
        p_at_k = hits / k
        scores.append(p_at_k)

        print(f"Query: {q}")
        print(f"Precision@{k}: {p_at_k:.2f}\n")

    print(f"Precision@{k} promedio: {float(np.mean(scores)):.2f}")
    return scores

# --- Construir Hybrid ---
# Asume que ya existen:
#   dense_ret = vector_store_hier.as_retriever(search_type="similarity", search_kwargs={"k": BASE_K})
#   bm25_ret  = BM25Retriever.from_documents(all_splits); bm25_ret.k = BASE_K

hybrid_ret = HybridUnionRetriever(dense_ret=dense_ret, bm25_ret=bm25_ret, k=BASE_K)

scores_hybrid = evaluate_retriever_precision(
    hybrid_ret,
    eval_queries,
    k=BASE_K,
    nombre=f"Hito 2 - HYBRID (BM25 + Dense, k={BASE_K})"
)

hybrid_precision = float(np.mean(scores_hybrid))
print("\nP@5 (HYBRID) =", hybrid_precision, " Delta =", hybrid_precision - baseline_precision)


NameError: name 'dense_ret' is not defined

In [ ]:
# ============================
# HITO 2 — HYBRID + RERANK (con TU clase RerankRetriever)
# ============================

# 0) Seguridad: cosas que deben existir
assert "bm25_ret" in globals(), "No existe bm25_ret. Ejecuta primero la celda de BM25."
assert "vector_store_hier" in globals(), "No existe vector_store_hier."
assert "RerankRetriever" in globals(), "No existe la clase RerankRetriever (la del CrossEncoder). Ejecuta esa celda primero."
assert "reranker" in globals(), "No existe 'reranker' (tu CrossEncoder). Ejecuta la celda donde creas CrossEncoder."
assert "BASE_K" in globals(), "No existe BASE_K."
assert "evaluate_retriever_precision" in globals(), "No existe evaluate_retriever_precision."
assert "eval_queries" in globals(), "No existe eval_queries."
assert "baseline_precision" in globals(), "No existe baseline_precision."

# 1) Dense (jerárquico)
dense_ret = vector_store_hier.as_retriever(
    search_type="similarity",
    search_kwargs={"k": BASE_K}
)

# 2) HYBRID simple que usa .invoke()
class HybridInvokeRetriever:
    def __init__(self, sparse_retriever, dense_retriever, k=5):
        self.sparse = sparse_retriever
        self.dense = dense_retriever
        self.k = k

    def invoke(self, query: str):
        docs_sparse = self.sparse.invoke(query)
        docs_dense  = self.dense.invoke(query)

        # merge + dedupe por (source + snippet)
        seen = set()
        merged = []
        for d in (docs_sparse + docs_dense):
            key = (str(d.metadata.get("source","")), d.page_content[:200])
            if key not in seen:
                seen.add(key)
                merged.append(d)

        return merged[: self.k]

    def get_relevant_documents(self, query: str):
        return self.invoke(query)

hyb_pool = HybridInvokeRetriever(bm25_ret, dense_ret, k=BASE_K)

# 3) Aplicar TU reranking encima del híbrido
# OJO: tu RerankRetriever usa fetch_k del base retriever, así que el pool debe traer más docs.
# Para eso, hacemos un pool "más grande" y luego el rerank deja top_k=BASE_K.

hyb_pool_big = HybridInvokeRetriever(bm25_ret, dense_ret, k=30)

retriever_hybrid_rerank = RerankRetriever(
    base_retriever=hyb_pool_big,
    cross_encoder=reranker,
    top_k=BASE_K,
    fetch_k=30
)

scores_hybrid_rerank = evaluate_retriever_precision(
    retriever_hybrid_rerank,
    eval_queries,
    k=BASE_K,
    nombre=f"Hito 2 - HYBRID + RERANK (k={BASE_K})"
)

hybrid_rerank_precision = float(np.mean(scores_hybrid_rerank))
print("\nP@5 (HYBRID+RERANK) =", hybrid_rerank_precision)
print("Delta vs baseline =", hybrid_rerank_precision - baseline_precision)


In [ ]:
import pandas as pd
import os

# asegurar carpeta de resultados
os.makedirs("results", exist_ok=True)

# guardar resultados finales del Hito 2
results_hito2 = pd.DataFrame([
    {
        "modelo": "Hito 2 - Hybrid + Rerank",
        "k": 5,
        "precision_at_5": 0.4666666666666666,
        "delta_vs_baseline": 0.20
    }
])

results_hito2.to_csv("results/hito2_hybrid_rerank.csv", index=False)

print(" Resultados guardados en results/hito2_hybrid_rerank.csv")


**CONCLUSION:**

La incorporación de un esquema de Hybrid Retrieval combinado con reranking mediante Cross-Encoder produjo una mejora importante en el desempeño del sistema de recuperación de información respecto al baseline definido en el Hito 1. Mientras el baseline jerárquico basado únicamente en similitud semántica alcanzó una precisión@5 de 0.27, la arquitectura Hybrid + Rerank logró una precisión@5 de 0.47, representando un incremento absoluto de +0.20.

Este resultado evidencia que la combinación de señales densas (embeddings semánticos), señales léxicas (BM25) y un modelo de reranking supervisado permite priorizar documentos más relevantes en las primeras posiciones del ranking, especialmente en consultas conceptuales.

In [ ]:
import pandas as pd
import os

os.makedirs("results", exist_ok=True)

results_comparison = pd.DataFrame([
    {"Modelo": "Hito 1 - Baseline (Hierarchical Similarity)", "Precision@5": 0.27, "Delta_vs_Baseline": 0.00},
    {"Modelo": "Hito 2 - MMR", "Precision@5": 0.20, "Delta_vs_Baseline": -0.07},
    {"Modelo": "Hito 2 - Query Processing (Expansion)", "Precision@5": 0.33, "Delta_vs_Baseline": 0.07},
    {"Modelo": "Hito 2 - Reranking (CrossEncoder)", "Precision@5": 0.33, "Delta_vs_Baseline": 0.07},
    {"Modelo": "Hito 2 - BM25", "Precision@5": 0.47, "Delta_vs_Baseline": 0.20},
    {"Modelo": "Hito 2 - Hybrid (BM25 + Dense)", "Precision@5": 0.27, "Delta_vs_Baseline": 0.00},
    {"Modelo": "Hito 2 - Hybrid + Rerank", "Precision@5": 0.47, "Delta_vs_Baseline": 0.20},
])

results_comparison.to_csv("results/comparacion_hito1_hito2.csv", index=False)

results_comparison


La Tabla anterior presenta la comparación de desempeño entre el baseline definido en el Hito 1 y las distintas técnicas de mejora evaluadas en el Hito 2, utilizando la métrica Precision@5. El baseline jerárquico basado en similitud semántica alcanzó una precisión@5 de 0.27, sirviendo como punto de referencia para evaluar el impacto de las técnicas avanzadas de recuperación.

Los resultados muestran que técnicas como Query Processing mediante expansión de consultas y Reranking con modelos Cross-Encoder producen mejoras moderadas (+0.07), concluyendo que la reformulación de consultas y el reordenamiento supervisado ayudan a priorizar documentos relevantes. Al contrario, el uso de MMR no mejora el desempeño, lo que permite concluir que la penalización por redundancia no es beneficiosa para documentos técnicos extensos.

El mayor incremento se observa al incorporar recuperación léxica mediante BM25, alcanzando una precisión@5 de 0.47 (+0.20). Sin embargo, la combinación directa de BM25 con embeddings densos (Hybrid) no genera mejoras adicionales. Finalmente, la arquitectura Hybrid + Rerank logra igualar el mejor desempeño observado, consolidándose como la solución más robusta al integrar múltiples señales de recuperación y un modelo de reordenamiento supervisado.

**11. Pipeline de Generación Aumentada (RAG-G)**

In [ ]:
!pip -q install -U llama-cpp-python==0.2.90

import os, textwrap
from llama_cpp import Llama

# Modelo GGUF
MODEL_URL = "https://huggingface.co/TheBloke/Llama-2-7B-Chat-GGUF/resolve/main/llama-2-7b-chat.Q4_K_M.gguf"
MODEL_PATH = "llama-2-7b-chat.Q4_K_M.gguf"

# Descargar modelo
!wget -q -O {MODEL_PATH} {MODEL_URL}
print("Descargado:", os.path.getsize(MODEL_PATH), "bytes")

# Cargar modelo
llama = Llama(
    model_path=MODEL_PATH,
    n_ctx=4096,
    n_threads=8,
    n_gpu_layers=35  # si no tienes GPU o falla, pon 0
)

def llama_generate(prompt, max_tokens=300, temperature=0.2, top_p=0.9):
    out = llama(
        prompt,
        max_tokens=max_tokens,
        temperature=temperature,
        top_p=top_p,
        stop=["</s>"]
    )
    return out["choices"][0]["text"]


In [ ]:
from typing import List
from langchain_core.documents import Document

# 1) Compat: retriever puede ser LangChain (.invoke) o BM25 (.get_relevant_documents)
def _retrieve_any(retriever, query: str):
    if hasattr(retriever, "invoke") and callable(getattr(retriever, "invoke")):
        return retriever.invoke(query)
    if hasattr(retriever, "get_relevant_documents") and callable(getattr(retriever, "get_relevant_documents")):
        return retriever.get_relevant_documents(query)
    raise AttributeError(f"Retriever sin método compatible: {type(retriever)}")

# 2) Contexto aumentado con metadatos
def format_docs_for_context(docs: List[Document], max_chars: int = 12000) -> str:
    blocks, total = [], 0
    for i, d in enumerate(docs, 1):
        meta = d.metadata or {}
        source = meta.get("source", meta.get("file_name", "unknown_source"))
        page = meta.get("page", meta.get("page_number", "NA"))
        chunk_id = meta.get("chunk_id", meta.get("id", f"chunk_{i}"))

        text = (d.page_content or "").strip()
        block = f"[DOC {i}] source={source} | page={page} | chunk_id={chunk_id}\n{text}\n"
        if total + len(block) > max_chars:
            break
        blocks.append(block)
        total += len(block)
    return "\n---\n".join(blocks)

# 3) Prompt anti-alucinación + citas
def build_prompt(question: str, context: str) -> str:
    return f"""
Eres un asistente experto en preservantes.
REGLAS:
- Responde SOLO usando el CONTEXTO.
- Si no hay evidencia suficiente, responde: "No encuentro evidencia suficiente en los documentos."
- No inventes datos.
- Cita con [DOC i] en cada afirmación importante.

PREGUNTA:
{question}

CONTEXTO:
{context}

RESPUESTA (en español, clara y estructurada):
"""

# 4) Pipeline final
def ask_rag_llama_cpp(question: str, retriever, k_retrieval: int = 10, k_context: int = 5, max_chars: int = 12000):
    docs = _retrieve_any(retriever, question) or []
    docs = docs[:k_retrieval]
    docs_ctx = docs[:k_context]

    context = format_docs_for_context(docs_ctx, max_chars=max_chars)
    prompt = build_prompt(question, context)

    answer = llama_generate(prompt, max_tokens=350, temperature=0.2, top_p=0.9)

    return {
        "question": question,
        "answer": answer,
        "docs_used": [
            {
                "source": (d.metadata or {}).get("source", (d.metadata or {}).get("file_name", "unknown_source")),
                "page": (d.metadata or {}).get("page", (d.metadata or {}).get("page_number", "NA")),
                "chunk_id": (d.metadata or {}).get("chunk_id", (d.metadata or {}).get("id", None)),
                "snippet": (d.page_content or "")[:250]
            }
            for d in docs_ctx
        ]
    }


In [ ]:
# ============================
# RAG-G con LLaMA GGUF
# - auto-detecta el retriever
# - genera respuestas con citas
# - guarda resultados (JSON/CSV)
# ============================

import json
import pandas as pd
from typing import List, Any, Dict

# ----------------------------
# 0) Validaciones mínimas
# ----------------------------
if "llama_generate" not in globals():
    raise RuntimeError("No encuentro llama_generate(). Ejecuta primero la celda del modelo GGUF (llama-cpp).")

# ----------------------------
# 1) Detectar automáticamente el mejor retriever
# ----------------------------
PREFERRED_RETRIEVER_NAMES = [
    # los más probables en tu notebook / estilo del curso
    "retriever_rerank", "retriever_mmr", "retriever_qproc",
    "retriever_hybrid", "hybrid_retriever",
    "bm25_ret", "bm25_retriever",
    "retriever_base", "retriever",
    "ensemble_retriever",
]

def _is_retriever(obj: Any) -> bool:
    return (
        (hasattr(obj, "invoke") and callable(getattr(obj, "invoke"))) or
        (hasattr(obj, "get_relevant_documents") and callable(getattr(obj, "get_relevant_documents")))
    )

def _pick_retriever_from_globals() -> Any:
    # 1) por nombre preferido
    for name in PREFERRED_RETRIEVER_NAMES:
        if name in globals() and _is_retriever(globals()[name]):
            return globals()[name], name

    # 2) fallback: primer objeto en globals() que parezca retriever
    candidates = []
    for k, v in globals().items():
        if k.startswith("_"):
            continue
        if _is_retriever(v):
            candidates.append((k, v))

    if not candidates:
        raise RuntimeError(
            "No encontré ningún retriever en el notebook. "
            "Asegúrate de haber creado al menos uno (vectorstore.as_retriever(), BM25Retriever, hybrid, etc.)."
        )

    # Heurística: si el nombre contiene 'rerank' o 'mmr' lo preferimos
    def score(name: str) -> int:
        s = 0
        n = name.lower()
        if "rerank" in n: s += 50
        if "mmr" in n: s += 40
        if "hybrid" in n: s += 30
        if "bm25" in n: s += 20
        if "base" in n: s += 10
        return s

    candidates.sort(key=lambda kv: score(kv[0]), reverse=True)
    return candidates[0][1], candidates[0][0]

retriever, retriever_name = _pick_retriever_from_globals()
print(f" Usando retriever detectado automáticamente: {retriever_name}")

# ----------------------------
# 2) Helpers robustos
# ----------------------------
def _retrieve_any(retriever: Any, query: str):
    if hasattr(retriever, "invoke") and callable(getattr(retriever, "invoke")):
        return retriever.invoke(query)
    if hasattr(retriever, "get_relevant_documents") and callable(getattr(retriever, "get_relevant_documents")):
        return retriever.get_relevant_documents(query)
    raise AttributeError(f"Retriever sin método compatible: {type(retriever)}")

def _get_meta(d):
    try:
        return d.metadata or {}
    except Exception:
        return {}

def format_docs_for_context(docs: List[Any], max_chars: int = 12000) -> str:
    blocks, total = [], 0
    for i, d in enumerate(docs, 1):
        meta = _get_meta(d)
        source = meta.get("source", meta.get("file_name", "unknown_source"))
        page = meta.get("page", meta.get("page_number", "NA"))
        chunk_id = meta.get("chunk_id", meta.get("id", f"chunk_{i}"))
        text = (getattr(d, "page_content", "") or "").strip()

        block = f"[DOC {i}] source={source} | page={page} | chunk_id={chunk_id}\n{text}\n"
        if total + len(block) > max_chars:
            break
        blocks.append(block)
        total += len(block)

    return "\n---\n".join(blocks)

def build_prompt(question: str, context: str) -> str:
    return f"""
Eres un asistente experto en preservantes y documentación técnica.
REGLAS ESTRICTAS:
- Responde SOLO usando el CONTEXTO.
- Si el contexto no contiene evidencia suficiente, responde exactamente:
  "No encuentro evidencia suficiente en los documentos."
- No inventes datos, cifras, límites ni nombres.
- Incluye citas [DOC i] en cada afirmación importante (definiciones, límites, efectos, recomendaciones).

PREGUNTA:
{question}

CONTEXTO:
{context}

RESPUESTA (en español, clara; usa bullets si aplica y termina con un resumen de 1-2 líneas):
"""

def ask_rag_llama_cpp(question: str, retriever: Any,
                      k_retrieval: int = 10, k_context: int = 5,
                      max_chars: int = 12000,
                      max_tokens: int = 350, temperature: float = 0.2, top_p: float = 0.9) -> Dict[str, Any]:
    docs = _retrieve_any(retriever, question) or []
    docs = docs[:k_retrieval]
    docs_ctx = docs[:k_context]

    context = format_docs_for_context(docs_ctx, max_chars=max_chars)
    prompt = build_prompt(question, context)

    answer = llama_generate(prompt, max_tokens=max_tokens, temperature=temperature, top_p=top_p)

    used = []
    for d in docs_ctx:
        meta = _get_meta(d)
        used.append({
            "source": meta.get("source", meta.get("file_name", "unknown_source")),
            "page": meta.get("page", meta.get("page_number", "NA")),
            "chunk_id": meta.get("chunk_id", meta.get("id", None)),
            "snippet": (getattr(d, "page_content", "") or "")[:250]
        })

    return {
        "retriever_used": retriever_name,
        "question": question,
        "answer": answer,
        "docs_used": used
    }

# ----------------------------
# 3) Ejecutar prueba + guardar resultados
# ----------------------------
QUESTIONS = [
    "¿Qué factores afectan la efectividad de los preservantes según los documentos?",
    "¿Qué riesgos o efectos adversos se mencionan sobre el uso de preservantes y en qué condiciones aparecen?",
    "¿Qué recomendaciones, límites o precauciones de uso se describen para preservantes en alimentos?"
]

results = []
for q in QUESTIONS:
    r = ask_rag_llama_cpp(q, retriever)
    results.append(r)
    print("\n" + "="*90)
    print("PREGUNTA:", r["question"])
    print("-"*90)
    print(r["answer"])
    print("\nFuentes usadas:")
    for i, d in enumerate(r["docs_used"], 1):
        print(f"  - [DOC {i}] {d['source']} | page {d['page']} | chunk {d['chunk_id']}")

# Guardar JSON
json_path = "results/rag_llama_results.json"
with open(json_path, "w", encoding="utf-8") as f:
    json.dump(results, f, ensure_ascii=False, indent=2)

# Guardar CSV (respuesta + fuentes)
rows = []
for r in results:
    sources = "; ".join([f"{d['source']}|p{d['page']}|{d['chunk_id']}" for d in r["docs_used"]])
    rows.append({
        "retriever_used": r["retriever_used"],
        "question": r["question"],
        "answer": r["answer"],
        "sources": sources
    })

df = pd.DataFrame(rows)
csv_path = "results/rag_llama_results.csv"
df.to_csv(csv_path, index=False, encoding="utf-8")

print("\n Listo. Archivos generados:")
print(" -", json_path)
print(" -", csv_path)


**CONCLUSION**

La implementación del pipeline de generación aumentada (RAG-G) permitió mejorar la calidad y trazabilidad de las respuestas generadas a partir de los pdfs cargados. Al integrar un esquema de recuperación con re-ranking y un modelo de lenguaje LLaMA (GGUF), se logró que las respuestas estuvieran explícitamente fundamentadas en el contexto recuperado, incorporando citas a las fuentes originales y reduciendo el riesgo de alucinaciones. El uso de un prompt con reglas estrictas —que obliga al modelo a responder únicamente con base en la evidencia disponible— resultó clave para mantener la fidelidad al contenido documental.

En términos de contenido, el sistema fue capaz de identificar de manera consistente los factores que afectan la efectividad de los preservantes, tales como condiciones de uso, características del alimento y limitaciones técnicas descritas en los documentos. Para estas preguntas, el pipeline produjo respuestas estructuradas, claras y con respaldo explícito en las fuentes, lo que evidencia una correcta alineación entre la etapa de recuperación y la etapa de generación.

Para las preguntas relacionadas con riesgos o efectos adversos, el modelo respondió indicando la ausencia de evidencia suficiente en los documentos disponibles. Este comportamiento que se busca en los sistemas RAG, ya que demuestra que el pipeline prioriza la veracidad y la evidencia documental por sobre la generación de respuestas especulativas. El sistema no solo mejora la calidad de las respuestas positivas, sino que también gestiona adecuadamente los casos de información incompleta.

Finalmente, las respuestas asociadas a recomendaciones, límites y precauciones de uso mostraron que el pipeline es capaz de sintetizar lineamientos técnicos a partir de múltiples fragmentos documentales, manteniendo coherencia y citabilidad. Los resultados confirman que la incorporación de un pipeline de generación aumentada aporta valor frente a un enfoque de recuperación simple, mejorando la confiabilidad, explicabilidad y utilidad práctica del sistema RAG en un campo técnico especializado como el de documentos cientificos